Another quick and dirty look at this data. I hand picked a few features I thought looked promising, did some normalisation , added in some categorcal one hot encoded ones, and ran simple linear regression over the data.

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
train_raw = pd.read_csv('../input/train.csv')
test_raw = pd.read_csv('../input/test.csv')

In [ ]:
f =['Id', 'SalePrice',
'LotArea' ,
'OverallQual',
'YearBuilt',
'1stFlrSF']

#    'MSZoning', 'LotConfig' , 'Neighborhood' , 'Condition1' , 'BldgType' ]

f_y = ['Id','SalePrice']
f_x = ['LotArea', 'OverallQual', 'YearBuilt', '1stFlrSF']#, 'MSZoning', 'LotConfig' , 'Neighborhood' , 'Condition1' , 'BldgType' ]

f_cat = ['MSZoning', 'LotConfig' , 'Neighborhood' , 'Condition1' , 'BldgType' ]

f_x2 = ['Id', 'LotArea', 'OverallQual', 'YearBuilt','PoolQC' ,'SaleCondition' , '1stFlrSF','MSZoning', 'LotConfig' , 'Neighborhood' , 'Condition1' , 'BldgType','Heating','Exterior1st' ,'CentralAir', 'Functional']

train_x = train_raw[f_x2]
train_y = train_raw[f_y]
test_x = test_raw[f_x2]

x_comb = pd.concat((train_x, test_x))
x_comb = pd.get_dummies(x_comb)


for col in f_x:
    mn = x_comb.loc[:,col].mean()
    sd = x_comb.loc[:,col].std()
    x_comb.loc[:,col] -= mn
    x_comb.loc[:,col] /= sd

test_x = x_comb[x_comb.Id > 1460]
train_x = x_comb[x_comb.Id < 1461]
train_x.drop(['Id'], axis=1, inplace=True)
test_x.drop(['Id'], axis=1, inplace=True)



#1460 train
#1461 on test

train_x, dev_x, train_y, dev_y = train_test_split(train_x, train_y, test_size=1, random_state=2)





In [ ]:
from sklearn import linear_model
reg = linear_model.Ridge(alpha=0.99)
reg.fit(train_x, train_y.SalePrice)
reg.coef_

In [ ]:
dev_y_pred = reg.predict(dev_x)
train_y_pred = reg.predict(train_x)
test_y = reg.predict(test_x)

In [ ]:
test_y = pd.Series(test_y.squeeze())
test_y[test_y < 0] *= -1
test_out = pd.concat([test_raw.Id, test_y], axis=1)
test_out.to_csv('lin_reg.csv', index=False, header=('Id','SalePrice'))

In [ ]:
print(np.sqrt(mean_squared_error(train_y.SalePrice, train_y_pred)) / 1000) 
print(np.sqrt(mean_squared_error(dev_y.SalePrice, dev_y_pred)) / 1000) 